In [3]:
from pyspark.sql.types import IntegerType
%load_ext autoreload
%autoreload 2

import pyspark
from delta import *

import sys

sys.path.append("..")  # Adjust path if needed
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *

from pyspark.sql.functions import col, lit, max as spark_max, length, row_number, explode

def create_spark_local_session():
    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
spark = create_spark_local_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
delta_sports = "../data/delta_sports/leagues/part-00000-0ee262ed-d010-4d83-912f-c464a7d3249d-c000.snappy.parquet"
delta_tmdb = "../data/delta_tmdb/trending/part-00000-2e2f77c7-c3dc-440a-80ff-bc0fb3811041-c000.snappy.parquet"

In [15]:
def read_parquet(spark, path):
    df = spark.read.parquet(path)
    df.show(5)
    return df

In [18]:
df=read_parquet(spark, delta_tmdb)

+-----+--------------------+-------------+-------+----------+-----------------+--------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+--------------------+
|adult|       backdrop_path|    genre_ids|     id|media_type|original_language|      original_title|            overview|popularity|         poster_path|release_date|               title|video|vote_average|vote_count|      ingestion_time|
+-----+--------------------+-------------+-------+----------+-----------------+--------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+--------------------+
|false|/fTrQsdMS2MUw00Rn...| [28, 80, 53]|1197306|     movie|               en|       A Working Man|Levon Cade left b...| 1086.1895|/xUkUZ8eOnrOnnJAf...|  2025-03-26|       A Working Man|false|       6.324|       387|2025-04-23 08:47:...|
|false|/nAxGnGHOsfzufThz...| [27, 28, 53]|12